In [18]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd=32
dropout = 0.2
n_layer = 6
n_head = 6
# ------------

In [2]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00


In [3]:
import os
import requests
import tiktoken
import numpy as np
input_file_path="Shakespeare.txt"
# download the tiny shakespeare dataset
data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
with open(input_file_path, 'w', encoding='utf-8') as f:
    f.write(requests.get(data_url).text)


In [4]:
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = f.read()
n = len(data)

In [5]:
print(n)

1115394


In [6]:
print(data[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
chars=sorted(list(set(data)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [9]:
print(encode('yeet'))
print(decode([63, 43, 43, 58]))

[63, 43, 43, 58]
yeet


In [10]:
import torch

In [11]:
new_data=torch.tensor(encode(data),dtype=torch.long)
print(new_data.shape,new_data.dtype)
print(new_data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
n=int(0.9*len(new_data))
train=new_data[:n]
test=new_data[n:]

In [13]:
torch.manual_seed(1337)


def get_batch(split):
  data = train if split=='train' else test
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([new_data[i:i+block_size] for i in ix])
  y=torch.stack([new_data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb=get_batch('train')

print('inputs')
print(xb)
print(xb.shape)
print('outputs')
print(yb)
print(yb.shape)

print('\n-------\n')

for b in range(batch_size):
  for t in range(block_size):
    context=xb[b,: t+1]
    target=yb[b,t]
    print(f"when input is {context} the target: {target}")

Streaming output truncated to the last 5000 lines.
when input is tensor([ 5, 57,  1, 52, 39, 51, 43,  6,  1, 42, 53, 58, 46,  1, 40, 43, 41, 53,
        51, 43,  1, 53, 44,  1, 58, 46, 47, 57, 12,  0,  0, 26, 27, 30, 32, 20,
        33, 25, 14, 17, 30, 24, 13, 26, 16, 10,  0, 35, 39, 56, 57,  1, 46, 39,
        60, 43,  1, 52, 53, 58,  1, 61, 39, 57, 58, 43, 42,  1, 47, 58,  6,  1,
        44, 53, 56,  1, 61, 39, 56, 56,  5, 42,  1, 46, 43,  1, 46, 39, 58, 46,
         1, 52, 53, 58,  6,  0, 14, 59, 58,  1, 40, 39, 57, 43, 50, 63,  1, 63,
        47, 43, 50, 42, 43, 42,  1, 59, 54, 53, 52,  1, 41, 53, 51, 54, 56, 53,
        51, 47, 57, 43,  0, 32, 46, 39, 58,  1, 61, 46, 47, 41, 46,  1, 46, 47,
        57,  1, 52, 53, 40, 50, 43,  1, 39, 52, 41, 43, 57, 58, 53, 56, 57,  1,
        39, 41, 46, 47, 43]) the target: 60
when input is tensor([ 5, 57,  1, 52, 39, 51, 43,  6,  1, 42, 53, 58, 46,  1, 40, 43, 41, 53,
        51, 43,  1, 53, 44,  1, 58, 46, 47, 57, 12,  0,  0, 26, 27, 30, 32, 2

In [14]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out


In [19]:
torch.manual_seed(1337)
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T= idx.shape
    # print(T)
    tok_emb=self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device) % block_size).unsqueeze(0)
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)
    if targets is None:
      loss=None
    else:
      B, T, C = logits.shape
      logits=logits.view(B*T,C)
      targets=targets.view(B * T)
      loss=F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond=idx[:,-block_size:]
      logits,loss=self(idx_cond)
      logits=logits[:,-1,:]
      probs=F.softmax(logits, dim=-1)
      idx_next=torch.multinomial(probs,num_samples=1)
      idx=torch.cat((idx,idx_next),dim=1)
    return idx



m=BigramLanguageModel()
logits,loss=m(xb,yb)
print(logits)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

tensor([[-0.3913, -0.3811,  0.5570,  ...,  0.1027, -0.1306, -0.0287],
        [ 0.5306, -0.3153, -0.5478,  ...,  0.1133,  1.4345,  0.7113],
        [ 0.7626,  0.2167, -0.3095,  ...,  1.3286,  1.0936,  0.2557],
        ...,
        [-0.6317,  0.2037,  0.2277,  ...,  0.1540, -0.4667, -0.4334],
        [-0.5946, -0.8423,  0.2819,  ..., -0.2332,  0.2058, -0.1993],
        [ 0.7200, -0.6012, -0.9567,  ...,  0.8404,  0.1058, -0.7697]],
       grad_fn=<ViewBackward0>)
tensor(4.3633, grad_fn=<NllLossBackward0>)

!AcVpRNItvPTgUCQXUhfkEfSq;
rKv-YG!rh-U P
BTJ!HUjOq
K.YhqmpXppDkqYjJPqPpi-rqBDFC$N,P3cuITC!PyIncORyXY


In [20]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [ ]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3662, val loss 4.3691


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long)

In [ ]:
print(context.shape)

Self Attention Mathematical Trick


In [ ]:
# xbow= torch.zeros((B,T,C))
# for b in range(B):
#   for t in range(T):
#     xprev=x[b,:t+1]
#     xbow[b,t]=torch.mean(xprev,0)

In [ ]:
x[0]

In [ ]:
wei = torch.tril(torch.ones(T,T))
wei= wei/wei.sum(1,keepdim=True)
xbow2= wei @ x
torch.allclose(xbow,xbow2,atol=1e-6)

In [ ]:
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))